In [1]:
import re
import numpy as np
import pandas as pd
from textblob import TextBlob
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#import flair
from flair.models import TextClassifier
from flair.data import Sentence

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import streamlit as st

In [3]:
#st.title('Chrome reviews')

In [4]:
'''#@st.cache
def load_data():
    df = pd.read_csv(r'D:\NextLabs\chrome_reviews.csv')
    return df'''

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 53-54: malformed \N character escape (<ipython-input-4-0faac9fde607>, line 4)

In [ ]:
#df = load_data()
'''uploaded_file = st.file_uploader("Choose a file")
if uploaded_file is not None:
    #read csv
    df=pd.read_csv(uploaded_file)'''

In [5]:
df = pd.read_csv(r'D:\NextLabs\chrome_reviews.csv')

In [6]:
df = df[["ID", "Text", "Star", "Thumbs Up"]]

In [7]:
print("Null data in the train dataset:\n", df.isnull().any())

Null data in the train dataset:
 ID           False
Text          True
Star         False
Thumbs Up    False
dtype: bool


In [8]:
df[df['Text'].isnull()]

,ID,Text,Star,Thumbs Up
5163,390986,NaN,5,0


In [9]:
df = df.dropna()

In [10]:
print("Null data in the train dataset:\n", df.isnull().any())

Null data in the train dataset:
 ID           False
Text         False
Star         False
Thumbs Up    False
dtype: bool


In [11]:
sid = SentimentIntensityAnalyzer()

In [12]:
df["compound_score"] = [sid.polarity_scores(text)['compound'] for text in df['Text'].values]

In [13]:
df.head(2)

,ID,Text,Star,Thumbs Up,compound_score
0,3886,This is very helpfull aap.,5,0,0.0000
1,3887,Good,3,2,0.4404


In [14]:
df['vader_score'] = [1 if x>0.05 else 0 for x in df['compound_score']]

In [15]:
flair_classifier = TextClassifier.load('en-sentiment')

2022-06-08 23:45:49,807 loading file C:\Users\anike\.flair\models\sentiment-en-mix-distillbert_4.pt


In [16]:
sentiment=[]
confidence=[]

for sentence in df['Text']:
    sentence = Sentence(sentence)
    flair_classifier.predict(sentence)
    sentiment.append(sentence.labels[0].value)
    confidence.append(sentence.labels[0].score)

In [17]:
df["sentiment"] = sentiment
df["confidence"] = confidence

In [18]:
df['flair_score'] = [1 if x == 'POSITIVE' else 0 for x in df['sentiment']]

In [19]:
df.head(2)

,ID,Text,Star,Thumbs Up,compound_score,vader_score,sentiment,confidence,flair_score
0,3886,This is very helpfull aap.,5,0,0.0000,0,POSITIVE,0.979062,1
1,3887,Good,3,2,0.4404,1,POSITIVE,0.931395,1


In [20]:
vader_accuracy, flair_accuracy, total = 0,0,0
for index, row in df.iterrows():
    if row['Star'] != 3:
        total+=1
        if (row['Star'] > 3 and row['flair_score'] ==1) or (row['Star'] < 3 and row['flair_score'] ==0):
            flair_accuracy +=1
        if (row['Star'] > 3 and row['vader_score'] ==1) or (row['Star'] < 3 and row['vader_score'] ==0):
            vader_accuracy +=1        
flair_accuracy = flair_accuracy/total
vader_accuracy = vader_accuracy/total

In [21]:
print("The accuracy using vader is ", vader_accuracy)
print("The accuracy using flair is ", flair_accuracy)

The accuracy using vader is  0.7277843601895735
The accuracy using flair is  0.831457345971564


The accuracy using Flair is the highest

In [23]:
df[['Text','Star','compound_score']][((df['Star']<3) & (df['compound_score']>0.35)) | ((df['Star']>3) & (df['compound_score']<-0.05))]

,Text,Star,compound_score
15,Very coming in real status. Thank you,1,0.3612
48,Unable to update won't load crashing at times ...,1,0.4404
58,Not good!,4,-0.4015
69,It's no update why?,5,-0.2960
101,Good,1,0.4404
...,...,...,...
7084,Very Bad service😠😞😠😞😠😞,5,-0.5849
7109,Trouble with getting proper access to webpages...,4,-0.9603
7147,I'm not able to update the app for some reason...,2,0.5080
7189,Frustration!!,4,-0.5696


In [26]:
df[['Text','Star','sentiment','confidence']][((df['Star']<3) & (df['flair_score']==1) & (df['confidence']>0.75)) | ((df['Star']>3) & (df['flair_score']==0) & (df['confidence']>0.75))]

,Text,Star,sentiment,confidence
15,Very coming in real status. Thank you,1,POSITIVE,0.991135
25,Rait crom,5,NEGATIVE,0.997250
27,Continual Awe snaps... Fix it,4,NEGATIVE,0.757084
30,Data consuming,1,POSITIVE,0.986192
34,Not bad,5,NEGATIVE,0.818031
...,...,...,...,...
7181,Try to update chrome not Working,5,NEGATIVE,0.999637
7182,It's osm brows anything,5,NEGATIVE,0.978813
7187,Ponraj,5,NEGATIVE,0.999956
7189,Frustration!!,4,NEGATIVE,0.999874
